In [2]:
import pandas as pd

train_df = pd.read_pickle('train.pkl')
test_df = pd.read_pickle('test.pkl')
val_df = pd.read_pickle('val.pkl')

In [3]:
import pandas as pd

flat_data = []
for entry in train_df:
    patient_info = {key: entry[key] for key in entry if key != 'history'}
    for record in entry['history']:
        flat_record = {**patient_info, **record}
        flat_data.append(flat_record)

train_df = pd.DataFrame(flat_data)



In [4]:
flat_data = []
for entry in test_df:
    patient_info = {key: entry[key] for key in entry if key != 'history'}
    for record in entry['history']:
        flat_record = {**patient_info, **record}
        flat_data.append(flat_record)

test_df = pd.DataFrame(flat_data)

In [5]:
flat_data = []
for entry in val_df:
    patient_info = {key: entry[key] for key in entry if key != 'history'}
    for record in entry['history']:
        flat_record = {**patient_info, **record}
        flat_data.append(flat_record)

val_df = pd.DataFrame(flat_data)

In [6]:
train_df.head()

patient_id  age gender  next_ckd  \
0    10000032   52      F         0   
1    10000032   52      F         0   
2    10000032   52      F         0   
3    10000032   52      F         0   
4    10000032   52      F         0   

                                           diagnoses  \
0  [Portal hypertension, Other ascites, Cirrhosis...   
1  [Portal hypertension, Other ascites, Cirrhosis...   
2  [Unspecified viral hepatitis C with hepatic co...   
3  [Portal hypertension, Other ascites, Cirrhosis...   
4  [Unspecified viral hepatitis C with hepatic co...   

                                       abnormal_labs  length_of_stay  \
0                                       [Creatinine]        0.786111   
1                                       [Creatinine]        0.786111   
2  [Albumin, Creatinine, Chloride, Potassium, Sod...        1.015278   
3                                       [Creatinine]        0.786111   
4  [Albumin, Creatinine, Chloride, Potassium, Sod...        1.015278   

   days_since_first  
0                 0  
1                 0  
2                50  
3                 0  
4                50

In [7]:
def aggregate_patient_data(group):
    diagnoses_combined = set(diagnosis for sublist in group['diagnoses'] for diagnosis in sublist)

    # Combine abnormal labs into a unique set
    abnormal_labs_combined = set(lab for sublist in group['abnormal_labs'] for lab in sublist)

    # Determine next_ckd (if any 1 exists in the group, label it as 1)
    next_ckd = 1 if group['next_ckd'].any() == 1 else 0
    num_visits = group.shape[0]

    # Create the aggregated data for the patient
    aggregated_data = {
        'patient_id': group['patient_id'].iloc[0],
        'age': group['age'].iloc[0],
        'gender': group['gender'].iloc[0],
        'is_ckd': next_ckd,
        'diagnoses': list(diagnoses_combined),
        'abnormal_labs': list(abnormal_labs_combined),
        'num_visits':num_visits
    }

    return pd.Series(aggregated_data)




In [8]:
aggregated_train_df = train_df.groupby('patient_id').apply(aggregate_patient_data).reset_index(drop=True)
aggregated_test_df = test_df.groupby('patient_id').apply(aggregate_patient_data).reset_index(drop=True)
aggregated_val_df = val_df.groupby('patient_id').apply(aggregate_patient_data).reset_index(drop=True)

<ipython-input-8-d41aa5138e5f>:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  aggregated_train_df = train_df.groupby('patient_id').apply(aggregate_patient_data).reset_index(drop=True)
<ipython-input-8-d41aa5138e5f>:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  aggregated_test_df = test_df.groupby('patient_id').apply(aggregate_patient_data).reset_index(drop=True)
<ipython-input-8-d41aa5138e5f>:3: Deprec

In [9]:
def create_patient_history(aggregated_df):
  aggregated_df['diagnoses_str'] = aggregated_df['diagnoses'].apply(lambda x: ' '.join(x) if isinstance(x, list) else str(x))
  aggregated_df['abnormal_labs_str'] = aggregated_df['abnormal_labs'].apply(lambda x: ' '.join(x) if isinstance(x, list) else str(x))

  aggregated_df['patient_history'] = "diagnoses:"+aggregated_df['diagnoses_str'] + " , " + \
                                      aggregated_df['abnormal_labs_str'] + " "+ "age:" + \
                                      aggregated_df['age'].astype(str) + " , " + "gender:" + \
                                      aggregated_df['gender'].astype(str) + " , " + "number of visits:"+ \
                                      aggregated_df['num_visits'].astype(str)
  return aggregated_df



In [10]:
aggregated_train_df = create_patient_history(aggregated_train_df).reset_index(drop=True)
aggregated_test_df = create_patient_history(aggregated_test_df).reset_index(drop=True)
aggregated_val_df = create_patient_history(aggregated_val_df).reset_index(drop=True)

In [ ]:
aggregated_train_df.to_pickle("aggregated_train_df.pkl")
aggregated_test_df.to_pickle("aggregated_test_df.pkl")
aggregated_val_df.to_pickle("aggregated_val_df.pkl")
